In [2]:
import pandas as pd
import openpyxl
import os
import glob
from pathlib import Path

## Load centralised Table

### Data cleaning

In [3]:
# load centralised table

#df_centralised = pd.read_csv("df_centralised.csv")

df_centralised =  pd.read_csv("df_centralised3.csv")

df_centralised.head()




,Year,Name,State,RegionName,SiteDescription,TrialCode,SowingDate,HarvestDate,Abandoned,Range,...,Soil_Nitrate Nitrogen,Soil_Organic Carbon,Soil_pH (CaCl2),Soil_pH (water),Soil_Phosphorous,Soil_Potassium (K),Soil_Reactive Iron,Soil_Sulphur,Soil_Texture,Soil_Zinc
0,2015,Wheat,NSW,N/E,Bellata,trial_719,2015-05-19,2015-11-24,False,1,...,10.55,0.676,6.75,7.7,23.0,200.0,25.58,4.58,5.0,0.718
1,2015,Wheat,NSW,N/E,Bellata,trial_719,2015-05-19,2015-11-24,False,1,...,10.55,0.676,6.75,7.7,23.0,200.0,25.58,4.58,5.0,0.718
2,2015,Wheat,NSW,N/E,Bellata,trial_719,2015-05-19,2015-11-24,False,1,...,10.55,0.676,6.75,7.7,23.0,200.0,25.58,4.58,5.0,0.718
3,2015,Wheat,NSW,N/E,Bellata,trial_719,2015-05-19,2015-11-24,False,1,...,10.55,0.676,6.75,7.7,23.0,200.0,25.58,4.58,5.0,0.718
4,2015,Wheat,NSW,N/E,Bellata,trial_719,2015-05-19,2015-11-24,False,1,...,10.55,0.676,6.75,7.7,23.0,200.0,25.58,4.58,5.0,0.718


In [4]:
df_centralised.shape

(110187, 84)

In [5]:
soil_impute_cols = ['Soil_Year',
       'Soil_Released', 'Soil_SoilTestID', 'Soil_MinDepth', 'Soil_MaxDepth',
       'Soil_Aluminium', 'Soil_Ammonium Nitrogen', 'Soil_Boron',
       'Soil_Calcium', 'Soil_Chloride', 'Soil_Conductivity (EC)',
       'Soil_Copper', 'Soil_Exchangeable Al', 'Soil_Exchangeable Ca',
       'Soil_Exchangeable K', 'Soil_Exchangeable Mg', 'Soil_Exchangeable Mn',
       'Soil_Exchangeable Na', 'Soil_Gravel', 'Soil_Magnesium',
       'Soil_Manganese', 'Soil_Nitrate Nitrogen', 'Soil_Organic Carbon',
       'Soil_pH (CaCl2)', 'Soil_pH (water)', 'Soil_Phosphorous',
       'Soil_Potassium (K)', 'Soil_Reactive Iron', 'Soil_Sulphur',
       'Soil_Texture', 'Soil_Zinc']

In [6]:
from sklearn.impute import KNNImputer

# KNN imputation

imputer = KNNImputer(n_neighbors=5, weights='uniform')
df_centralised[soil_impute_cols] = imputer.fit_transform(df_centralised[soil_impute_cols])

In [7]:
pd.DataFrame(df_centralised.isnull().sum(), columns=['Number of missing values'])

,Number of missing values
Year,0
Name,0
State,0
RegionName,0
SiteDescription,0
...,...
Soil_Potassium (K),0
Soil_Reactive Iron,0
Soil_Sulphur,0
Soil_Texture,0


### drop around 30 records from soil data with missing values

In [8]:
df_centralised.dropna(subset=['Soil_Year',
       'Soil_Released', 'Soil_SoilTestID', 'Soil_MinDepth', 'Soil_MaxDepth',
       'Soil_Aluminium', 'Soil_Ammonium Nitrogen', 'Soil_Boron',
       'Soil_Calcium', 'Soil_Chloride', 'Soil_Conductivity (EC)',
       'Soil_Copper', 'Soil_Exchangeable Al', 'Soil_Exchangeable Ca',
       'Soil_Exchangeable K', 'Soil_Exchangeable Mg', 'Soil_Exchangeable Mn',
       'Soil_Exchangeable Na', 'Soil_Gravel', 'Soil_Magnesium',
       'Soil_Manganese', 'Soil_Nitrate Nitrogen', 'Soil_Organic Carbon',
       'Soil_pH (CaCl2)', 'Soil_pH (water)', 'Soil_Phosphorous',
       'Soil_Potassium (K)', 'Soil_Reactive Iron', 'Soil_Sulphur',
       'Soil_Texture', 'Soil_Zinc'], inplace=True)



### split into two df (one with sentinel data one without)

In [9]:
# show whole row
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


sentinel_col = ['B2', 'B3', 'B4', 'B5', 'B8', 'B8A', 'B11', 'ndvi', 'evi', 'evi2',
       'gndvi', 'avi', 'savi', 'osavi', 'vari', 'ndmi', 'msi', 'sipi', 'ndre1',
       'ndre2', 'dvi', 'cirededge', 'ccci']
df_centralised_no_sentinel = df_centralised.drop(columns=sentinel_col)


pd.DataFrame(df_centralised_no_sentinel.isnull().sum(), columns=['Number of missing values'])

,Number of missing values
Year,0
Name,0
State,0
RegionName,0
SiteDescription,0
TrialCode,0
SowingDate,0
HarvestDate,0
Abandoned,0
Range,0


In [10]:
pd.DataFrame(df_centralised.isnull().sum(), columns=['Number of missing values'])

,Number of missing values
Year,0
Name,0
State,0
RegionName,0
SiteDescription,0
TrialCode,0
SowingDate,0
HarvestDate,0
Abandoned,0
Range,0


In [11]:
df_centralised.to_csv("df_centralised_cleaned.csv", index=False)
df_centralised.to_parquet("df_centralised_cleaned.parquet", index=False)

In [12]:
df_centralised_sentinel = df_centralised.copy()
df_centralised_sentinel.dropna(subset=sentinel_col, inplace=True)
pd.DataFrame(df_centralised_sentinel.isnull().sum(), columns=['Number of missing values'])
df_centralised_sentinel.shape

(37045, 84)

In [13]:
df_centralised_sentinel.columns

Index(['Year', 'Name', 'State', 'RegionName', 'SiteDescription', 'TrialCode',
       'SowingDate', 'HarvestDate', 'Abandoned', 'Range', 'Row', 'CultivarID',
       'Harvest Length', 'Harvest Width', 'Kg/Plot', 'T.Max_avg', 'Smx_avg',
       'T.Min_avg', 'Smn_avg', 'Rain_avg', 'Srn_avg', 'Evap_avg', 'Sev_avg',
       'Radn_avg', 'Ssl_avg', 'VP_avg', 'Svp_avg', 'RHmaxT_avg', 'RHminT_avg',
       'B2', 'B3', 'B4', 'B5', 'B8', 'B8A', 'B11', 'ndvi', 'evi', 'evi2',
       'gndvi', 'avi', 'savi', 'osavi', 'vari', 'ndmi', 'msi', 'sipi', 'ndre1',
       'ndre2', 'dvi', 'cirededge', 'ccci', 'Soil_TrialCode', 'Soil_Year',
       'Soil_Released', 'Soil_SoilTestID', 'Soil_MinDepth', 'Soil_MaxDepth',
       'Soil_Aluminium', 'Soil_Ammonium Nitrogen', 'Soil_Boron',
       'Soil_Calcium', 'Soil_Chloride', 'Soil_Conductivity (EC)',
       'Soil_Copper', 'Soil_Exchangeable Al', 'Soil_Exchangeable Ca',
       'Soil_Exchangeable K', 'Soil_Exchangeable Mg', 'Soil_Exchangeable Mn',
       'Soil_Exchangeable

In [14]:
df_centralised_sentinel.to_csv("df_centralised_sentinel_cleaned2.csv", index=False)
df_centralised_sentinel.to_parquet("df_centralised_sentinel_cleaned2.parquet", index=False)